<a href="https://colab.research.google.com/github/perelloliver/BuyFromEU-Reddit-Scraper/blob/main/BFEU_Scraper_Public.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Enter required details for API access
client_id and client_secret can be acquired from the reddit project, ask Oliver to add you

user_agent_input is our project name + your username, just a message so admins can contact you if anything goes wrong.

In [ ]:
client_id_input = "" # @param {"type":"string","placeholder":"Input your client ID"}


In [ ]:
client_secret_input = "" # @param {"type":"string","placeholder":"Input your client secret"}


In [ ]:
user_agent_input = "buyfromEU_from_u/your_username" # @param {"type":"string","placeholder":"Input your user agent"}


# 2. Imports / Setup client

In [ ]:
!pip install praw
!pip install mistralai

import praw
import pandas as pd
import json
import os
import ast
from mistralai import Mistral


In [ ]:
client = praw.Reddit(
    client_id=client_id_input,
    client_secret=client_secret_input,
    user_agent=user_agent_input
)
subreddit_client = client.subreddit("buyfromeu")

# Set search words and flairs to check

In [ ]:
search_terms = ['alternative', 'alternatives', 'instead', 'swap', 'transfer', 'switch', 'switching', 'check out', 'suggest']

In [ ]:
flairs = ['Suggested Product or Service', 'Alternative Product or Service']

# 3. Extract posts relevant to flairs and search terms and save as .csv.
### *Optionally, download from files sidebar. Recommended in case you lose progress.*

In [ ]:
from pydantic import BaseModel

class Post(BaseModel):
  title: str
  post_text: str
  comments: list
  url: str

In [ ]:
def handle_comments(submission):
  comments = []
  comment_ids = submission.comments
  comment_ids.replace_more(limit=50)
  for comment in comment_ids.list():
    comments.append(comment.body)
  return comments

In [ ]:
submissions = []

for term in search_terms:
  for submission in subreddit_client.search(term):
    print("Working on submission...")
    title = submission.title
    post_text = submission.selftext
    comments = handle_comments(submission)
    url = submission.url

    submissions.append(Post(title=title, post_text=post_text, url=url, comments=comments))


In [ ]:
for flair in flairs:
  print(flair)
  for submission in subreddit_client.search(f'flair:"{flair}"'):
    print("Handling submission...")
    title = submission.title
    post_text = submission.selftext
    comments = handle_comments(submission)
    url = submission.url

    submissions.append(Post(url=url, comments=comments, post_text=post_text, title=title))


In [ ]:
extracted_data = pd.DataFrame([x.dict() for x in submissions])

In [ ]:
extracted_data.to_csv("bfeu_extracted.csv")

# 4. Input Mistral API key (requires paid tier due to data load)

In [ ]:
mistral_api_key = "" # @param {"type":"string"}

# 5. Use Mistral to extract original products and alternatives from each thread.

In [ ]:
data = pd.read_csv('bfeu_extracted.csv')

In [ ]:
!pip install mistralai
from mistralai import Mistral

In [ ]:
mistral_client = Mistral(api_key=mistral_api_key)
model = 'mistral-large-latest'

In [ ]:
system_prompt = """

You are a helpful assistant tasked to extract data from reddit threads.
You never fabricate data.
You dilligently review the reddit threads at hand and extract the data as instructed.

Take your time to ensure all the data you extract is correct. Do not fabricate data. If there is no suitable data for you to output, output an empty string or N/A.

"""

task_prompt = """

Identify the product being discussed, the American version and European alternative from the reddit thread provided.
Take your time to ensure all the data you extract is correct. Do not fabricate data. If there is no suitable data for you to output, output an empty string or N/A.

Use the following format in JSON:

"product_type": the product type mentioned in the reddit threat i.e video streaming service
"american_products": the american companies or products mentioned in the reddit thread e.g Netflix, Amazon Prime
"european_products" the european products mentioned in the reddit thread e.g BritBox, RaiPlay

Reddit thread: {thread}

"""

In [ ]:
dataset = pd.DataFrame()

In [ ]:
misformatted = []

In [ ]:
for index, row in data.iterrows():
  try:
    thread_data = (
        f"title: {row['title']}\n"
        f"post_text: {row['post_text']}\n"
        f"comments: {row['comments']}\n"
    )

    messages = [
        {
            "role": "system",
            "content": system_prompt,
        },

        {
            "role": "user",
            "content": task_prompt.format(thread=thread_data)
            }
    ]

    chat_response = mistral_client.chat.complete(
          model = model,
          messages = messages,
          response_format = {
              "type": "json_object",
          }
    )

    output = json.loads(chat_response.choices[0].message.content)
    dataset = pd.concat([dataset, pd.DataFrame([output])], ignore_index=True)
  except Exception as e:
    print(e)
    misformatted.append(output)
    continue

# 6. Save to CSV to clean, format and wrangle later
### *Optionally, download from files sidebar. Recommended in case you lose progress.*

In [ ]:
dataset.to_csv('extracted_products.csv')

# Check the dataset out

In [ ]:
dataset

# 8. Clean and format the dataset
Concatenate repeats, drop miscellaneous columns, format data for consistency.

We will output two datasets: one with aggregated fields under the same product type (so all american and european coca cola alternatives are grouped, for example) and one where there is a single line per product.

This is pure choice to allow flexibility with methods of reviewing and entering data.

There could be some improvements made in terms of execution here, but it works well.

### General formatting and creating our aggregated dataset.

In [ ]:
dataset = pd.read_csv('extracted_products.csv')

In [ ]:
# Drop old columns
clean_dataset = dataset[['product_type', 'american_products', 'european_products']]

In [ ]:
def flatten(lst):
  """Helper function to flatten lists"""
  for item in lst:
      if isinstance(item, list):
          yield from flatten(item)
      else:
          yield item

In [ ]:
# Replace NaN values with "N/A"
clean_dataset['american_products'] = clean_dataset['american_products'].apply(lambda x: x if isinstance(x, list) else (x if isinstance(x, str) else "N/A"))
clean_dataset['european_products'] = clean_dataset['european_products'].apply(lambda x: x if isinstance(x, list) else (x if isinstance(x, str) else "N/A"))


In [ ]:
# Convert string representations of lists to actual lists
clean_dataset['american_products'] = clean_dataset['american_products'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith('[') else x)
clean_dataset['european_products'] = clean_dataset['european_products'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith('[') else x)


In [ ]:
# Flatten lists if needed
clean_dataset['american_products'] = clean_dataset['american_products'].apply(lambda x: list(flatten([x])) if isinstance(x, list) else [x])
clean_dataset['european_products'] = clean_dataset['european_products'].apply(lambda x: list(flatten([x])) if isinstance(x, list) else [x])

# Ensure all values in american_products and european_products are lists
clean_dataset['american_products'] = clean_dataset['american_products'].apply(lambda x: x.split(', ') if isinstance(x, str) else x)
clean_dataset['european_products'] = clean_dataset['european_products'].apply(lambda x: x.split(', ') if isinstance(x, str) else x)

# Normalize the strings: convert to lowercase and strip whitespace
clean_dataset['american_products'] = clean_dataset['american_products'].apply(lambda x: [item.lower().strip() for item in x] if isinstance(x, list) else x)
clean_dataset['european_products'] = clean_dataset['european_products'].apply(lambda x: [item.lower().strip() for item in x] if isinstance(x, list) else x)

# Explode the lists to separate rows
exploded_dataset = clean_dataset.explode('american_products').explode('european_products')


In [ ]:
# Group by product_type, aggregating unique values in american_products and european_products
clean_formatted_dataset = exploded_dataset.groupby('product_type', as_index=False).agg({
    'american_products': lambda x: list(set(x)) if "n/a" not in x else "N/A",
    'european_products': lambda x: list(set(x)) if "n/a" not in x else "N/A"
})

clean_formatted_dataset

### Now let's make our single item <> row dataset.

In [ ]:
exploded_dataset.drop_duplicates(inplace=True)

In [ ]:
# Split comma-separated strings into lists, then re-explode.
exploded_dataset['american_products'] = exploded_dataset['american_products'].apply(lambda x: x.split(', ') if isinstance(x, str) else x)
exploded_dataset['european_products'] = exploded_dataset['european_products'].apply(lambda x: x.split(', ') if isinstance(x, str) else x)
exploded_dataset = exploded_dataset.explode('american_products').explode('european_products')

In [ ]:
exploded_dataset.drop_duplicates()

# Check your datasets out!
*Run this and expand to view.*

In [ ]:
exploded_dataset.head()

In [ ]:
clean_formatted_dataset.head()

# 8. Save the final datasets to csv - make sure to download them from the sidebar!

In [ ]:
clean_formatted_dataset.to_csv("bfeu_formatted_final.csv")

In [ ]:
exploded_dataset.to_csv("bfeu_single_line_final.csv")